# Homework 2: Learning about Datasets

In [1]:
#Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd

## 1. Write code to load in the dataset

## Loading Example Dataset 
Decided to focus on the RCP 8.5 dataset which is the most extreme emissions scenario for Santa Barbara area (9 cells centered on 34.4208° N, 119.6982° W). 

Downloaded Monthly Averages:
-Daily Mean Preciptation rate
-Daily Maximum Temperature
-Daily Minimum Temperature

In [2]:
# xarray was used to load in the data
sb_mean_precip = xr.open_dataset('sb_mean_precip_data.nc')
sb_mean_max_temp = xr.open_dataset('sb_mean_max_temp.nc')
sb_mean_min_temp = xr.open_dataset('sb_mean_min_temp.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/danielkerstan/Desktop/EDS_211/notebook/sb_mean_precip_data.nc'

In [ ]:
#view metadata
sb_mean_max_temp

## Date and Geospatial Format
Metadata indicates that date is in datetime64 format and geospatial information in stored as lat and lon(float64). 

In [3]:
sb_county = gpd.read_file('tl_2019_06083_faces/tl_2019_06083_faces.shp')

xmin, ymin, xmax, ymax = sb_county.total_bounds

In [4]:
sb_county.total_bounds

array([-120.734382,   33.411024, -118.962728,   35.114678])

In [5]:
url = "https://ds.nccs.nasa.gov/thredds/dodsC/bypass/NEX-DCP30/nex-quartile/rcp85/r1i1p1.ncml"

nex=xr.open_dataset(url)

In [6]:
nex

<xarray.Dataset>
Dimensions:            (lat: 3105, lon: 7025, bnds: 2, time: 1128)
Coordinates:
  * lat                (lat) float64 24.06 24.07 24.08 ... 49.91 49.92 49.93
  * lon                (lon) float64 235.0 235.0 235.0 ... 293.5 293.5 293.5
  * time               (time) datetime64[ns] 2006-01-16T12:00:00 ... 2099-12-...
Dimensions without coordinates: bnds
Data variables: (12/15)
    lat_bnds           (lat, bnds) float64 ...
    lon_bnds           (lon, bnds) float64 ...
    time_bnds          (time, bnds) float64 ...
    ens-avg_pr         (time, lat, lon) float32 ...
    ens-avg_tasmax     (time, lat, lon) float32 ...
    ens-avg_tasmin     (time, lat, lon) float32 ...
    ...                 ...
    quartile50_pr      (time, lat, lon) float32 ...
    quartile50_tasmax  (time, lat, lon) float32 ...
    quartile50_tasmin  (time, lat, lon) float32 ...
    quartile75_pr      (time, lat, lon) float32 ...
    quartile75_tasmax  (time, lat, lon) float32 ...
    quartile75_tasmin  (time, lat, lon) float32 ...
Attributes: (12/39)
    CDI:                            Climate Data Interface version 1.5.5 (htt...
    parent_experiment:              historical
    parent_experiment_id:           historical
    parent_experiment_rip:          r1i1p1
    creation_date:                  Mon Jul 23 02:05:29 PDT 2012
    CDO:                            Climate Data Operators version 1.5.5 (htt...
    ...                             ...
    title:                          800m Downscaled NEX CMIP5 Climate Project...
    contact:                        Dr. Rama Nemani: rama.nemani@nasa.gov, Dr...
    model_id:                       NEX-quartile
    references:                     BCSD method: Wood AW, Maurer EP, Kumar A,...
    DOI:                            http://dx.doi.org/10.7292/W0WD3XH4
    experiment:                     RCP8.5

In [7]:
# Create logical masks for lat and lon variables
mask_lon = (nex.lon >= xmin+360) & (nex.lon <= xmax+360)
mask_lat = (nex.lat >= ymin) & (nex.lat <= ymax)

# Apply lat/lon masks to the ERSST field, then calculate averages over the lat and lon dimensions
nex_sb_max=nex.where(mask_lon & mask_lat).mean("lon").mean("lat")

RuntimeError: NetCDF: Access failure